In [24]:
using StaticArrays,BenchmarkTools,Random,Flux,DelimitedFiles,LinearAlgebra
using InvertedIndices
using MachineLearningPotential

# Symmetry Functions

Time to write and test the basics of SymmFunctions for ML potentials

In [2]:
struct RadialType2{T} <: RadialSymmFunction{T}
    eta::T
    r_cut::T
end


In [3]:
atoms = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]*18.8973*0.36258

positions = [SVector{3}(p[i] for i in 1:3) for p in atoms]
dis2mat = get_distance2_mat(positions)
test_symm_func = RadialType2{Float64}(0.001,11.338)

RadialType2{Float64}(0.001, 11.338)

In [4]:
function calc_one_symm_function(r2_ij,eta,r_cut)
    if r2_ij != 0 && r2_ij < r_cut^2 
        g_ij = exp(-eta*(r2_ij)) * cutoff_function( sqrt(r2_ij)/r_cut )

        return g_ij
    else
        return 0.
    end
end

function calc_symm_functions(dist2_matrix,index,symmfunc::RadialType2)
    eta = symmfunc.eta
    r_cut = symmfunc.r_cut
    
    gvec = calc_one_symm_function.(dist2_matrix[index,:],eta,r_cut)
    return sum(gvec)   
end
function calc_symm_functions(dist2_matrix,index,symmfunc::RadialType2,g_min,g_max)
    eta = symmfunc.eta
    r_cut = symmfunc.r_cut
    gvec = calc_one_symm_function.(dist2_matrix[index,:],eta,r_cut)
    g_unscaled = sum(gvec)

    return (g_unscaled - g_min)/(g_max - g_min)   
end

calc_symm_functions (generic function with 1 method)

In [5]:
[calc_symm_functions(dis2mat,i,test_symm_func) for i in 1:55]

55-element Vector{Float64}:
 14.640666117305633
 11.966607067855739
 11.966607067541357
 11.966607067163583
 11.966607068360855
 11.966607068035858
 11.96660706756069
 11.966607067316641
 11.966607067815215
 11.96660706776792
  ⋮
  5.6995369746290425
  7.455371452799304
  5.699536974721713
  7.455371453127722
  7.455371452781338
  7.455371453231261
  7.455371452614264
  7.455371452215095
  5.699536975557825

In [20]:
[calc_symm_functions(dis2mat,i,test_symm_func,0.450564942,15.523345849) for i in 1:55]

55-element Vector{Float64}:
 0.9414388269065572
 0.7640290266879376
 0.76402902666708
 0.7640290266420167
 0.7640290267214493
 0.7640290266998875
 0.7640290266683626
 0.7640290266521712
 0.764029026685249
 0.7640290266821113
 ⋮
 0.34824177867478656
 0.4647321920234493
 0.34824177868093475
 0.46473219204523813
 0.46473219202225735
 0.46473219205210736
 0.46473219201117283
 0.46473219198469007
 0.3482417787364064

Require benchmarking concerning the use of Not(i) vs if ||

In [17]:
@benchmark calc_symm_functions($dis2mat,1,test_symm_func)

BenchmarkTools.Trial: 10000 samples with 192 evaluations.
 Range (min … max):  515.880 ns … 26.183 μs  ┊ GC (min … max): 0.00% … 97.37%
 Time  (median):     699.716 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   702.297 ns ±  1.018 μs  ┊ GC (mean ± σ):  6.09% ±  4.10%

    █▂▅                       ▃▃▂▄                              
  ▆▇███▇▅▃▂▁▁▁▂▂▂▂▂▂▂▁▁▁▁▁▂▃▆▇█████▇▆▃▂▂▂▂▃▃▃▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁ ▃
  516 ns          Histogram: frequency by time          916 ns <

 Memory estimate: 1008 bytes, allocs estimate: 3.

In [28]:
function test_calc_one_symm_func(r2_ij,eta,r_cut)
    if r2_ij < r_cut^2
        g_ij = exp(-eta*(r2_ij)) * cutoff_function( sqrt(r2_ij)/r_cut )
    else
        g_ij = 0
    end
    return g_ij
end
function test_calc_one_symm_func(r2_ij,symmfunc)
    if r2_ij < symmfunc.r_cut^2
        g_ij = exp(-symmfunc.eta*(r2_ij)) * cutoff_function( sqrt(r2_ij)/r_cut )
    else
        g_ij = 0
    end
    return g_ij
end
function test_calc_symm_functions(dist2_mat,index,symmfunc)
    eta = symmfunc.eta
    r_cut = symmfunc.r_cut
    g_tot = 0
    for j in (1:55)[Not(index)]
        g_tot += calc_one_symm_function(dist2_mat[index,j],eta,r_cut)
    end
    return g_tot
end

test_calc_symm_functions (generic function with 1 method)

In [29]:
@benchmark test_calc_symm_functions($dis2mat,1,test_symm_func)

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.462 μs … 772.709 μs  ┊ GC (min … max): 0.00% … 99.09%
 Time  (median):     5.008 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.126 μs ±  17.919 μs  ┊ GC (mean ± σ):  7.13% ±  2.43%

   ▂▇█▄                                                        
  ▃████▆▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▂▄▆▇█▆▄▃▂▂▂▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  4.46 μs         Histogram: frequency by time        9.13 μs <

 Memory estimate: 8.94 KiB, allocs estimate: 273.

# Angular Symmetry Function

In [6]:
function angular_measure(a,b,c)
    r_ab,r_ac,r_bc = sqrt(distance2(a,b)),sqrt(distance2(a,c)),sqrt(distance2(b,c))

    θ = (a - b)⋅(a - c)/(r_ab*r_ac)
    
    return θ,r_ab,r_ac,r_bc
end

angular_measure (generic function with 1 method)

In [8]:
θ,r_ab,r_bc,r_bc = angular_measure(positions[1:3]...)

(0.4472135955245632, 4.575916481041805, 4.575916481145328, 4.811403321032106)

In [ ]:
# function calc_one_symm_function(r2_ij,r2_ik,r2_jk,r_cut,eta,lambda,zeta)
#     d_cut=r_cut^2
#     if r2_ij == 0. || r2_ik == 0 || r2_jk == 0 || r2_ij<d_cut || r_ik<d_cut || r_jk < d_cut 
#         g = 0
#     else
        
#     end

#     return g
# end